In [2]:
import requests
import pandas as pd

In [3]:
# load .env and fetch api key
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ.get('riot_api_key')

In [4]:
def get_puuid(summonerId:str=None, gameName:str=None, tagLine:str=None, region:str='europe'):
    '''Given summonerId or gameName and tagLine of Riot Account returns the puuid of that account

    Arguments:
        str: summonerId, defualts to None.
        str: gameName, defualts to None.
        str: tagLine, defualts to None.
        str: region, defualts to 'europe'.

    Returns:
        str: puuid
    '''

    if summonerId is not None:
        # region tag is a silly little goose
        if region == 'europe':
            region = 'euw1'
        root = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        request = root + endpoint + summonerId + '?api_key=' + api_key
        response = requests.get(request)
    else:
        if gameName is None or tagLine is None:
            raise Exception("Please input gameName and tagLine")

        request = f'https://{region}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}'
        response = requests.get(request)

    if not response.status_code == 200:
         raise Exception("Response Error: {}".format(response.status_code))
    puuid = response.json()['puuid']

    return puuid

In [5]:
# testing get_puuid
# gameName1 = 'SONDER'
# tagLine1 = 'mafk1'
# summonerId1 = '0qDuJrMmD3ht2azWwkEZ1iAAuZ3wpWfcrKsTNHe9yIYoEfY'
# print("puuid for SONDER#mafk1: " + get_puuid(summonerId=summonerId1))
# print("puuid equal using summonerId vs gameName and tagLine?", get_puuid(gameName=gameName1, tagLine=tagLine1) == get_puuid(summonerId=summonerId1))

In [6]:
def get_ranked_match_history(puuid:str=None, region:str='europe', n_matches:int=20):
    '''Given puuid of Riot Account, returns a list of match IDs as strings

    Arguments:
        str: puuid, defualts to None.
        str: region, defualts to 'europe'.
        int: n_matches, defualts to 20 and must be between 0 and 100 (inclusive).

    Returns:
        list: match IDs as strings
    '''
    
    if puuid is None:
        raise Exception('Please provide a puuid in string format.')
    if n_matches > 100:
        raise Exception('Amount of matches cannot exceed 100.')
    if n_matches < 0:
        raise Exception('Amount of matches must be at least 0.')

    root = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}'
    request = root + endpoint + '/ids?type=ranked&start=0&count=' + str(n_matches) + '&api_key=' + api_key
    response = requests.get(request)

    if not response.status_code == 200:
         raise Exception("Response Error: {}".format(response.status_code))

    return response.json()
    


In [7]:
# testing get_ranked_match_history
# print("Amount of matches is 20?", len(get_ranked_match_history(puuid='smKXivxW6VCPxTu9NyeWVNNObhayvp7njxu8EmGlOxQ94ttoGDGYp7a9-cgB68kxXUj2EhIWysGfVg')) == 20)
# get_ranked_match_history(puuid=get_puuid(summonerId=summonerId1))

In [8]:
def get_match_data(matchId:str=None, region='europe'):
    '''Given League of Legends Match ID, returns a pandas dataframe of useful match data

    Arguments:
        str: matchId, defualts to None.
        str: region, defualts to 'europe'.

    Returns:
        dataframe: match data
    '''

    root = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/{matchId}'
    request = root + endpoint + '?api_key=' + api_key
    response = requests.get(request)

    if not response.status_code == 200:
         raise Exception("Response Error: {}".format(response.status_code))

    df = pd.DataFrame.from_dict(response.json())

    return df
    

# What info does get_match_data dataframe contain?

## InfoDto

- gameVersion - contains patch info
- participants - list of ParticipantDto

## ParticipantDto

- championName -  name of champion played
- championTransform - used for Kayn transformation, which can either be 0 (None), 1 (Slayer) or 2 (Assassin)
- item0
- item1
- item2
- item3
- item4
- item5
- item6
- lane - lane played
- perks - perksDto

## PerksDto

- statPerks -  PerkStatsDto
- styles - list of PerkStyleDto

## PerkStatsDto

- defense
- flex
- offense

## PerkStyleDto

- description
- selections - list of PerkStyleSelectionDto
- style

## PerkStyleSelectionDto

- perk
- var1
- var2
- var3

## TeamDto

- win

In [9]:
# champion = get_match_data(matchId='EUW1_6906634163')['info']['participants'][0]['championId']
# win = get_match_data(matchId='EUW1_6906634163')['info']['participants'][0]['win']

# print(champion)
# print(win)

In [10]:
# test_data = {
#     "match_id": [],
#     "champion": [],
#     "win": []
# }
# test_df = pd.DataFrame(test_data)

# print(test_df)

In [11]:
import time

c = {
    "match_id": [],
    "champion": [],
    "win": []
}

gathered_data = pd.DataFrame(c)

#print(gathered_data)

def gather_los_dataos(matches):
    count = 0
    for match in matches:
        if not (match in gathered_data['match_id'].value_counts().keys()):
            if count % 100 == 0 and count > 0: # make sure we don't exceed max number of API calls per 2 minutes
                time.sleep(120)
            info = get_match_data(matchId=match)['info']
            participants = info['participants']
            count += 1
            if not info['queueId'] == 420:
                continue
            for participant in participants:
                champion = participant['championId']
                win = participant['win']
                gathered_data.loc[len(gathered_data.index)] = [match, champion, win]


# tests                
#start_time = time.time()
#gather_los_dataos(['EUW1_6906634163', 'EUW1_6906634163'])
#gather_los_dataos(get_ranked_match_history(puuid=get_puuid(gameName='Dunkvern', tagLine='EUW')))
#print('Time elapsed: %s' % (time.time() - start_time))
#print(gathered_data)

In [12]:
#print(gathered_data['match_id'].value_counts())

In [13]:
def get_challenger(region:str='europe'):
    '''Given a region, returns the puuids of the challenger ladder.

    Arguments:
        str: region

    Returns:
        str[]: challenger ladder puuids
    '''

    # region tag is a silly little goose
    if region == 'europe':
        region = 'euw1'
    request = f'https://{region}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}'
    response = requests.get(request)

    if not response.status_code == 200:
         raise Exception("Response Error: {}".format(response.status_code))

    puuids = []
    summonerids = pd.DataFrame.from_dict(pd.DataFrame.from_dict(response.json()['entries']))['summonerId']
    count = 0 
    for summoner in summonerids:
        print(count)
        count += 1
        if len(puuids) % 99 == 0 and len(puuids) > 0: # make sure we don't exceed max number of API calls per 2 minutes
            time.sleep(120)
        puuids.append(get_puuid(summonerId=summoner))

    return puuids

def get_grandmaster(region:str='europe'):
    '''Given a region, returns the puuids of the challenger ladder.

    Arguments:
        str: region

    Returns:
        str[]: challenger ladder puuids
    '''

    # region tag is a silly little goose
    if region == 'europe':
        region = 'euw1'
    request = f'https://{region}.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}'
    response = requests.get(request)

    if not response.status_code == 200:
         raise Exception("Response Error: {}".format(response.status_code))

    puuids = []
    summonerids = pd.DataFrame.from_dict(pd.DataFrame.from_dict(response.json()['entries']))['summonerId']
    count = 0 
    for summoner in summonerids:
        print(count)
        count += 1
        if len(puuids) % 99 == 0 and len(puuids) > 0: # make sure we don't exceed max number of API calls per 2 minutes
            time.sleep(120)
        puuids.append(get_puuid(summonerId=summoner))

    return puuids

def get_master(region:str='europe'):
    '''Given a region, returns the puuids of the challenger ladder.

    Arguments:
        str: region

    Returns:
        str[]: challenger ladder puuids
    '''

    # region tag is a silly little goose
    if region == 'europe':
        region = 'euw1'
    request = f'https://{region}.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}'
    response = requests.get(request)

    if not response.status_code == 200:
         raise Exception("Response Error: {}".format(response.status_code))

    puuids = []
    summonerids = pd.DataFrame.from_dict(pd.DataFrame.from_dict(response.json()['entries']))['summonerId']
    count = 0 
    for summoner in summonerids:
        print(count)
        count += 1
        if len(puuids) % 99 == 0 and len(puuids) > 0: # make sure we don't exceed max number of API calls per 2 minutes
            time.sleep(120)
        puuids.append(get_puuid(summonerId=summoner))

    return puuids

def get_apex_ladder(region:str='europe'):
    print('Getting Challenger ladder...')
    challengers = get_challenger(region)
    time.sleep(120)
    print('Getting Grandmaster ladder...')
    grandmasters = get_grandmaster(region)
    time.sleep(120)
    print('Getting Master ladder...')
    masters = get_master(region)
    return challengers + grandmasters + masters

In [14]:
#print(len(get_apex_ladder()))

In [15]:
c = {
    "match_id": [],
    "champion": [],
    "win": []
}

gathered_data = pd.DataFrame(c)

def ultimate_data_collector(region:str='europe'):
    print('Getting ladder...')
    apex_ladder = get_apex_ladder()
    #apex_ladder = get_challenger()
    print('Ladder loaded!')
    print('Waiting for API limit...')
    time.sleep(120)
    count = 0

    all_matches = []
    print('Gathering matches...')
    for account in apex_ladder:
        if count % 100 == 0 and count > 0:
            time.sleep(120)
        count += 1
        # gather_los_dataos(get_ranked_match_history(puuid=account, n_matches=1))
        all_matches += get_ranked_match_history(puuid=account, n_matches=3)
        print(count)
    print('Processing matches...')
    #print(all_matches)
    time.sleep(120)
    gather_los_dataos(all_matches)
        

    print('Making CSV...')
    gathered_data.to_csv('DATA.csv')
    print('DONE!')
    

In [16]:
ultimate_data_collector()

Getting ladder...
Getting Challenger ladder...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


KeyboardInterrupt: 

In [ ]:
# lst = ['EUW1_6961705416', 'EUW1_6961154409', 'EUW1_6961124219', 'EUW1_6961072228', 'EUW1_6960961460', 'EUW1_6960892721', 'EUW1_6959602355', 'EUW1_6958211129', 'EUW1_6958134871', 'EUW1_6958024350', 'EUW1_6957403916', 'EUW1_6956979328', 'EUW1_6956879354', 'EUW1_6955889925', 'EUW1_6955823277', 'EUW1_6955800316', 'EUW1_6955438830', 'EUW1_6955294503', 'EUW1_6955251451', 'EUW1_6955194793', 'EUW1_6961902514', 'EUW1_6961373253', 'EUW1_6960496892', 'EUW1_6960469715', 'EUW1_6960433870', 'EUW1_6960395355', 'EUW1_6960254532', 'EUW1_6960229124', 'EUW1_6960201433', 'EUW1_6960138096', 'EUW1_6960106172', 'EUW1_6960095482', 'EUW1_6960065341', 'EUW1_6959900916', 'EUW1_6959853873', 'EUW1_6959802047', 'EUW1_6959752960', 'EUW1_6959711611', 'EUW1_6959660713', 'EUW1_6959619558', 'EUW1_6957572857', 'EUW1_6957511188', 'EUW1_6957485330', 'EUW1_6957233242', 'EUW1_6957190822', 'EUW1_6957135817', 'EUW1_6957042251', 'EUW1_6957010167', 'EUW1_6956970347', 'EUW1_6956934213', 'EUW1_6956896931', 'EUW1_6956834246', 'EUW1_6956258604', 'EUW1_6956118131', 'EUW1_6956086884', 'EUW1_6956046629', 'EUW1_6955994298', 'EUW1_6955959075', 'EUW1_6955919022', 'EUW1_6955886980', 'EUW1_6961930012', 'EUW1_6961881219', 'EUW1_6961811596', 'EUW1_6961347041', 'EUW1_6961317199', 'EUW1_6961100560', 'EUW1_6961065547', 'EUW1_6961020089', 'EUW1_6960974591', 'EUW1_6960920940', 'EUW1_6960844167', 'EUW1_6960795678', 'EUW1_6960724776', 'EUW1_6960677687', 'EUW1_6960613523', 'EUW1_6960565954', 'EUW1_6960528369', 'EUW1_6960240957', 'EUW1_6960201924', 'EUW1_6960172625', 'EUW1_6961857060', 'EUW1_6961789370', 'EUW1_6961734202', 'EUW1_6961403467', 'EUW1_6960769463', 'EUW1_6960403929', 'EUW1_6960112320', 'EUW1_6960087554', 'EUW1_6960060256', 'EUW1_6959679666', 'EUW1_6959644708', 'EUW1_6959567194', 'EUW1_6959315819', 'EUW1_6959270893', 'EUW1_6959102955', 'EUW1_6959079804', 'EUW1_6958174176', 'EUW1_6958149673', 'EUW1_6958117064', 'EUW1_6958062015']
# print(len(set(lst)))

# df = pd.read_csv("DATA.csv")
# print(df['match_id'].value_counts())

# lst2 = df['match_id'].to_list()

# s = set(lst2)

# temp = [x for x in lst if x not in s]
# print(temp)

In [17]:
def extract_json(json, key):
    items = []

    def extract(json, arr, key):
        if isinstance(json, dict):
            for k, v in json.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(json, list):
            for item in json:
                extract(item, arr, key)

        return arr
    
    values = extract(json, items, key)
    return values

In [18]:
champions = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champion-summary.json'

champions_json = requests.get(champions).json()

champion_ids = extract_json(champions_json, 'id')
champion_names = extract_json(champions_json, 'name')

champion_dict = dict(map(lambda i, j: (i,j), champion_ids, champion_names))

In [19]:
df_with_names = pd.read_csv('DATA.csv', index_col=0)

df_with_names = df_with_names.replace(champion_dict)

In [21]:
for name in champion_names:
    temp = df_with_names[df_with_names['champion'].isin([name])]
    temp.to_csv('championdata/' + name + '.csv', index=False)

In [24]:
(df_with_names['champion'].sort_values().drop_duplicates()).to_csv('Champions.csv', index=False)